In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.combine import SMOTEENN

In [2]:
# Set parameters
n_splits = 5
output_file = f'model_rf={n_splits}.bin'

In [3]:
# Data preparation
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')


In [4]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [5]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)
df.churn = (df.churn == 'yes').astype(int)


In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice',
    'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies',
    'contract', 'paperlessbilling', 'paymentmethod'
]


In [23]:
def train_rf(df_train, y_train):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    # Apply SMOTEENN to balance the data
    smote_enn = SMOTEENN(random_state=1)
    X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)

    # Train the Random Forest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=1)
    model.fit(X_train_resampled, y_train_resampled)

    return dv, model

In [24]:
print('Doing validation with Random Forest')

Doing validation with Random Forest


In [25]:

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
scores = []
fold = 0

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train_rf(df_train, y_train)
    dicts_val = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(dicts_val)
    y_pred = model.predict_proba(X_val)[:, 1]

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

    print(f'AUC on fold {fold} is {auc}')
    fold += 1

AUC on fold 0 is 0.8331966151454968
AUC on fold 1 is 0.8434499569593432
AUC on fold 2 is 0.8212261689745134
AUC on fold 3 is 0.8159373601074376
AUC on fold 4 is 0.8378482789997176


In [26]:
print('Validation results:')
print(f'AUC: {np.mean(scores):.3f} ± {np.std(scores):.3f}')

Validation results:
AUC: 0.830 ± 0.010


In [27]:
# Training the final Random Forest model
print('Training the final Random Forest model')

Training the final Random Forest model


In [28]:
dv, model = train_rf(df_full_train, df_full_train.churn.values)
dicts_test = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(dicts_test)
y_pred_test = model.predict_proba(X_test)[:, 1]

y_test = df_test.churn.values
auc_test = roc_auc_score(y_test, y_pred_test)

print(f'AUC on test set: {auc_test:.3f}')

AUC on test set: 0.837


In [29]:
# Save the Random Forest model
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

print(f'The Random Forest model is saved to {output_file}')

The Random Forest model is saved to model_rf.bin
